#### CORD 19  - Group 4
* For this project we have metadata, pdf_json and pmc_json file
* We need to combine json file into one csv with reference of metadata

In [1]:
import pandas as pd
import numpy as np
import glob
import json
import re
import csv
import os
from collections import defaultdict
from utils import *

In [2]:
root_path = '/scratch/midway2/reshmask/'

In [3]:
metadata  = pd.read_csv(root_path+'metadata.csv')

/project2/msca/ivy2/software2/install/Anaconda3-2019.10/envs/ML1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# metadata['cord_uid']

In [5]:
path_to_json = root_path+'document_parses/'
pdf_json     = glob.glob(f'{path_to_json}/pdf_json/*.json', recursive=True)
pmc_json     = glob.glob(f'{path_to_json}/pmc_json/*.json', recursive=True)

### EDA for metadata

In [6]:
# metadata.columns
# test = pdf_json[1:10]

In [7]:
#number of null abstracts
print ('Number of missing abstracts:', metadata[pd.isnull(metadata['abstract'])].shape[0])
print ('% of missing abstracts:', (metadata[pd.isnull(metadata['abstract'])].shape[0]/metadata.shape[0])*100)

Number of missing abstracts: 58970
% of missing abstracts: 28.790711980588117


#### pdf and pmc concerns - if you have pdf then you don't need to worry about PMC json files

Whenever pdf json is null - we will not have body_text of a paper

In [8]:
## Assumption that every pdf json file has body_text - we should atleast get
print ('Number of missing pdf json (body_text):', metadata[pd.isnull(metadata['pdf_json_files'])].shape[0])
print ('% of missing pdf json (body_text):', (metadata[pd.isnull(metadata['pdf_json_files'])].shape[0]/metadata.shape[0])*100)

Number of missing pdf json (body_text): 119577
% of missing pdf json (body_text): 58.38065061052714


In [9]:
metadata_pdf = metadata[pd.notnull(metadata['pdf_json_files'])]

In [10]:
## Assumption that every pdf json file has body_text - we should atleast get
print ('Number of missing pmc json:', metadata[pd.isnull(metadata['pmc_json_files'])].shape[0])
print ('% of missing pmc json:', (metadata[pd.isnull(metadata['pmc_json_files'])].shape[0]/metadata.shape[0])*100)
pmc_not_pdf = metadata[(pd.isnull(metadata['pdf_json_files'])) & (pd.notnull(metadata['pmc_json_files']))].shape[0]
print ('% of pmc json available when pdf json is missing:', (pmc_not_pdf/metadata.shape[0])*100)

Number of missing pmc json: 138831
% of missing pmc json: 67.78096209898303
% of pmc json available when pdf json is missing: 1.9284943585437184


In [11]:
metadata_pmc_not_pdf = metadata[(pd.isnull(metadata['pdf_json_files'])) & (pd.notnull(metadata['pmc_json_files']))]

#### Comparison between pdf json and pmc json folder files with metadata

#### PDF

In [12]:
sha_pdf = [name.replace('/scratch/midway2/reshmask/document_parses//pdf_json/', '').replace('.json', '') for name in pdf_json]

In [13]:
print ('Total pdf json files:', len(sha_pdf))
inter_pdf = list(set(sha_pdf).intersection(set(metadata['sha'].values)))
print ('% of pdf json available in metadata:',(len(inter_pdf)/len(sha_pdf))*100)

Total pdf json files: 90361
% of pdf json available in metadata: 89.38369429289185


#### PMC

In [14]:
id_pmc = [name.replace('/scratch/midway2/reshmask/document_parses//pmc_json/', '').replace('.xml.json', '') for name in pmc_json]

In [15]:
print ('Total pmc json files:', len(id_pmc))
inter_pmc = list(set(id_pmc).intersection(set(metadata_pmc_not_pdf['pmcid'].values)))
print ('pmc json is available where pdf is null when match in metadata:',len(inter_pmc))
print ('% of pmc json available where pdf is null when match in metadata:',(len(inter_pmc)/len(id_pmc))*100)

Total pmc json files: 65992
pmc json is available where pdf is null when match in metadata: 3950
% of pmc json available where pdf is null when match in metadata: 5.985574008970785


## Get body text along with abstract
   * intersection pdf json files - only pdf json files which is available in metadata_pdf
   * pmc json while where pdf is null & pmc json files which is available in metadata_pmc_not_pdf

In [17]:
abstract_body_pdf = getALLdataFrame(inter_pdf, metadata_pdf,'pdf', '/scratch/midway2/reshmask/document_parses//pdf_json/')

In [25]:
# abstract_body_pdf.head()

In [19]:
abstract_body_pmc = getALLdataFrame(inter_pmc, metadata, 'pmc', '/scratch/midway2/reshmask/document_parses//pmc_json/')

In [24]:
# abstract_body_pmc.head()

# Analysis Plan

## Logic 1 = studying only body text
    * data set 1 -  when you have abstract & body_text == logic 1
    * data set 2 -  when you have only body text, no abstract == logic 1
    * body_text_abstract = data set 1 + data set 2

## Logic2 = studying only abstract
    * data set 3 -  when you have only abstract, no body_text == logic 2
    * abstract_only = data set 3

## Discard papers where both body text and abstract are missing

### descriptive analysis - combine 

## Prepare Data for logic 1

In [26]:
abstract_body_scrp = pd.concat([abstract_body_pdf, abstract_body_pmc])

In [27]:
abstract_body_scrp['abstract'] = abstract_body_scrp['abstract'].apply(lambda x : np.nan if x is '' else x)
abstract_body_scrp['body_text'] = abstract_body_scrp['body_text'].apply(lambda x : np.nan if x is '' else x)

In [28]:
abstract_body =  abstract_body_scrp[(pd.notnull(abstract_body_scrp['body_text'])) | (pd.notnull(abstract_body_scrp['abstract']))]

### get missing abstracts which is not in pdfs, pmcs

In [29]:
metadata_abstract = metadata[pd.notnull(metadata['abstract'])]
metadata_abstract['is_sha'] = metadata_abstract['sha'].apply(lambda x :  1 if x in inter_pdf else 0)
metadata_abstract = metadata_abstract[metadata_abstract['is_sha'] ==0]
metadata_abstract['is_pmc'] = metadata_abstract['pmcid'].apply(lambda x :  1 if x in inter_pmc else 0)

/project2/msca/ivy2/software2/install/Anaconda3-2019.10/envs/ML1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
metadata_abstract = metadata_abstract[metadata_abstract['is_pmc'] ==0]

In [31]:
# sel_cols = ['paper_id', 'doi', 'abstract', 'body_text', 'authors', 'title', 'journal', 'publish_time', 'url', 'source_x']
sel_cols = ['cord_uid', 'paper_id', 'doi', 'abstract', 'body_text',  'authors', 'title', 'journal', 'publish_time', 'url', 'source_x', 'institution', 'settlement', 'region', 'country']
metadata_abstract.rename(columns={'sha': 'paper_id'}, inplace=True)
metadata_abstract['paper_id'] = metadata_abstract[['paper_id', 'pmcid']].apply(lambda x : x['pmcid'] if pd.isnull(x['paper_id']) else x['paper_id'], axis=1)

In [32]:
metadata_abstract['body_text']   = np.nan
metadata_abstract['institution'] = np.nan
metadata_abstract['settlement']  = np.nan
metadata_abstract['region']      = np.nan
metadata_abstract['country']     = np.nan

In [33]:
metadata_abstract_sel = metadata_abstract[sel_cols]

In [34]:
metadata_abstract_sel.shape

(77827, 15)

In [35]:
all_data = pd.concat([abstract_body, metadata_abstract_sel])

In [36]:
all_data.shape

(162545, 15)

In [39]:
for col in all_data.columns:
    all_data[col] = all_data[col].apply(lambda x : x.replace(',', ' ') if pd.notnull(x) else x)
    all_data[col] = all_data[col].apply(lambda x : x.replace('\n', ' ') if pd.notnull(x) else x)
    all_data[col] = all_data[col].apply(lambda x : x.replace('$', ' ') if pd.notnull(x) else x)

In [43]:
all_data.to_csv('cord19_V2.csv', index=False)